The purpose of this notebook is to calculate the power produced by a decay chain after the production of a given nuclide. Specifically, the power will be calculated for $t = 10^{n}$ seconds where $n \in \mathbb{Z}, -10 \le n \le 10$

In [1]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy #https://docs.juliahub.com/SymPy/ 
using PDFIO
using Unzip
using Interpolations
using Downloads
#plotlyjs()
data_dir = ("C:\\Cross-Section-Data\\")
cross_section_dir = data_dir
parent_dir = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\"

WebIO._IJuliaInit()

"C:\\Users\\engin\\Documents\\GitHub\\Energy\\"

In [3]:
decay_chain_dir = parent_dir * "ExportedData\\all_beta_decay_chains\\"
decay_chain_files = readdir(decay_chain_dir)
all_decay_chains = [CSV.read(decay_chain_dir * file, DataFrame)
                    for file in decay_chain_files]
all_decay_chains = [rename!(df, :Column1 => :Parent) for df in all_decay_chains]

3192-element Vector{DataFrame}:
 4×4 DataFrame
 Row │ Parent   e-Folding Time (seconds)  Average beta-decay energy  Daughter 
     │ String7  Float64                   Float64                    String7  
─────┼────────────────────────────────────────────────────────────────────────
   1 │ 100Ag                   173.989                         5.69  100Cd
   2 │ 100Cd                    70.8363                        5.69  100In
   3 │ 100In                     8.10795                       5.69  100Sn
   4 │ 100Sn                     1.70238                       5.69  100Sb
 3×4 DataFrame
 Row │ Parent   e-Folding Time (seconds)  Average beta-decay energy  Daughter 
     │ String7  Float64                   Float64                    String7  
─────┼────────────────────────────────────────────────────────────────────────
   1 │ 100Cd                    70.8363                        5.69  100In
   2 │ 100In                     8.10795                       5.69  100Sn
   3 │ 100Sn  

I derived the decay rate of the n-th generation nuclide in a decay chain here:

https://www.overleaf.com/project/627613def6b848465d85e5bb

https://github.com/MarcosP7635/Math-for-Energy/blob/main/main.tex

file:///C:/Users/engin/Downloads/Math_for_Energy(4).pdf 

In [4]:
factor(t, e_folding_time) = 1 - exp(t / -e_folding_time)

factor (generic function with 1 method)

In [5]:
a_prod(e_folding_times, t) = prod([factor(t, e_folding_times[k])
                                   for k in 2:length(e_folding_times)])

a_prod (generic function with 1 method)

In [6]:
r(e_fold_times, t) = (a_prod(e_fold_times, t) * exp(t / -e_fold_times[1])
                      /
                      -e_fold_times[1])


r (generic function with 1 method)

In [7]:
λ_0, λ_1, λ_2, t = @variables λ_0, λ_1, λ_2, t
expected_output = (1 - exp(t / -λ_1)) * (1 - exp(t / -λ_2)) * exp(t / -λ_0) / (-λ_0)

(-(1 - exp(t / (-λ_1)))*(1 - exp(t / (-λ_2)))*exp(t / (-λ_0))) / λ_0

In [8]:
e_folding_times = [λ_0, λ_1, λ_2]
r(e_folding_times, t)
Test.@test r(e_folding_times, t) - expected_output == 0

Test Passed
  Expression: r(e_folding_times, t) - expected_output == 0
   Evaluated: 0 == 0

It works! :) 

The average beta decay energy is given in keV. 

In [9]:
joules_per_keV = Rational(uconvert(@u_str("J"), 1 * @u_str("keV")) / @u_str("J"))

337//2103388558093277156

Avogadro's number

In [10]:
N_A = 6.02214076e23
conversion_factor = Rational(joules_per_keV * N_A)

6475021031458043//67108864

Now we calculate the power in watts

In [11]:
power_per_mole(decay_rate, decay_energy) = decay_rate * decay_energy * conversion_factor

power_per_mole (generic function with 1 method)

In [12]:
power_per_mole(e_folding_times, decay_energies, t) = sum([
    power_per_mole(r(e_folding_times[1:i], t), decay_energies[i])
    for i in 1:length(decay_energies)])

power_per_mole (generic function with 2 methods)

In [13]:
E_0, E_1, E_2, t = @variables E_0, E_1, E_2, t
expected_output = conversion_factor * sum([
    E_0 * exp(t / -λ_0) / (-λ_0),
    (1 - exp(t / -λ_1)) * E_1 * exp(t / -λ_0) / (-λ_0),
    (1 - exp(t / -λ_1)) * (1 - exp(t / -λ_2)) * E_2 * exp(t / -λ_0) / (-λ_0)
])

((-6475021031458043//67108864)*E_0*exp(t / (-λ_0))) / λ_0 + ((-6475021031458043//67108864)*E_1*(1 - exp(t / (-λ_1)))*exp(t / (-λ_0))) / λ_0 + ((-6475021031458043//67108864)*E_2*(1 - exp(t / (-λ_1)))*(1 - exp(t / (-λ_2)))*exp(t / (-λ_0))) / λ_0

In [14]:
decay_energies = [E_0, E_1, E_2]
Test.@test Symbolics.simplify(power_per_mole(e_folding_times, decay_energies, t) 
            - expected_output) == 0

Test Passed
  Expression: Symbolics.simplify(power_per_mole(e_folding_times, decay_energies, t) - expected_output) == 0
   Evaluated: 0 == 0

In [17]:
x = [power_per_mole(df[!,"e-Folding Time (seconds)"], df[!,"Average beta-decay energy"], 2)
for df in all_decay_chains];

It takes about .1 seconds to calculate the power per mole for the entire dataframe for a given time. 

In [27]:
exponents_for_time_array = -10:20/1000:10
collect(exponents_for_time_array)

1001-element Vector{Float64}:
 -10.0
  -9.98
  -9.96
  -9.94
  -9.92
  -9.9
  -9.88
  -9.86
  -9.84
  -9.82
  -9.8
  -9.78
  -9.76
   ⋮
   9.78
   9.8
   9.82
   9.84
   9.86
   9.88
   9.9
   9.92
   9.94
   9.96
   9.98
  10.0

In [31]:
time_array = [BigFloat(10)^x for x in exponents_for_time_array]

1001-element Vector{BigFloat}:
 1.000000000000000000000000000000000000000000000000000000000000000000000000000003e-10
 1.047128548050898505549645783823682271757193893433574137820153812424537637016191e-10
 1.0964781961431828604257317209264159331313714105011702911985913060102831306613e-10
 1.148153621496884066480156540139268838588675605873626387416861363160599339507005e-10
 1.202264434617413102533306847068551134853580025789314084263608408653746937282328e-10
 1.25892541179416618056939287182433126845996218444738646667433027351526766150798e-10
 1.318256738556404729589129642168660706457656431654852798000801526451019756623137e-10
 1.38038426460288664458362703761266589785155477230948719984049693714804016050564e-10
 1.445439770745927984903979184458196052506877843919927508765240153411912508988761e-10
 1.513561248436207171938059987874779303581263483786977823789873399913939914213657e-10
 1.584893192461110892181946192724010648088344291738932269688212001583469968929013e-10
 1.659586907437563078013118

In [46]:
power_time_series = [[power_per_mole(df[!,"e-Folding Time (seconds)"], 
                    df[!,"Average beta-decay energy"], time)
                    for time in time_array] for df in all_decay_chains]; 

3192-element Vector{Vector{BigFloat}}:
 [-3.155380343474517773182881336006113448165850785347324068054552557712315167057752e+06, -3.155380343474642235671033654867567158340905399751105784074443920158714968006779e+06, -3.155380343474772563895539631572338731654789515706651136319548685149302456066908e+06, -3.155380343474909034300036898644014406878884507075396273202646251332909590485755e+06, -3.155380343475051936356550313134062173222115908609594602247477887030515639001058e+06, -3.155380343475201573179501004584918370016457647869283928507138250117618158045092e+06, -3.155380343475358262168652653911759955663705479065718304234679947013955001334634e+06, -3.155380343475522335682358973827213178383928896776895891261954511067545521413381e+06, -3.155380343475694141742540278565942591845518022105660834169621507543389632929895e+06, -3.155380343475874044772884724885105221545757185179426897888627326563089617773437e+06  …  -3.600997774746101807994342119012069870744806164672691307644846301039068684845985e-164

In [54]:
zero_gen_isotopes = [df[1, "Parent"] for df in all_decay_chains]
power_time_series_dict = Dict([])
stop = length(power_time_series)
for i in 1:stop
        power_time_series_dict[zero_gen_isotopes[i]] = power_time_series[i]
end
power_time_series_dict

Dict{Any, Any} with 3192 entries:
  "113Sn" => BigFloat[-19.1026, -19.1026, -19.1026, -19.1026, -19.1026, -19.102…
  "243Cf" => BigFloat[-587251.0, -587251.0, -587251.0, -587251.0, -587251.0, -5…
  "168Eu" => BigFloat[-1.90269e+09, -1.90269e+09, -1.90269e+09, -1.90269e+09, -…
  "162Sm" => BigFloat[-1.4094e+08, -1.4094e+08, -1.4094e+08, -1.4094e+08, -1.40…
  "11N"   => BigFloat[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0,…
  "57V"   => BigFloat[-1.08725e+09, -1.08725e+09, -1.08725e+09, -1.08725e+09, -…
  "208Po" => BigFloat[-4.16099, -4.16099, -4.16099, -4.16099, -4.16099, -4.1609…
  "174Ir" => BigFloat[-4.81695e+07, -4.81695e+07, -4.81695e+07, -4.81695e+07, -…
  "144Gd" => BigFloat[-1.41886e+06, -1.41886e+06, -1.41886e+06, -1.41886e+06, -…
  "80Sr"  => BigFloat[-59664.3, -59664.3, -59664.3, -59664.3, -59664.3, -59664.…
  "33Ar"  => BigFloat[-2.19965e+09, -2.19965e+09, -2.19965e+09, -2.19965e+09, -…
  "220Pa" => BigFloat[-4.48174e+14, -4.48197e+14, -4.48221e+14, -4.48246e+1